# iminuit and an external minimizer

This is a topical tutorial. If you are new to iminuit, you should go through the basic tutorial first. 

It is possible to use another minimizer to find the minimum of a function, and then use iminuit to compute the error matrix. We will demonstrate this with a maximum-likelihood fit of a normal distribution, which is carried out with scipy.optimize.minimize. iminuit is then used to compute the parameter uncertainties.

In [1]:
from iminuit import Minuit
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize

In [2]:
# normally distributed data
x = norm(0, 1).rvs(1000)

# negative log-likelihood for a normal distribution
def nll(par):
    return -np.sum(norm(par[0], par[1]).logpdf(x))


# minimize nll with scipy.optimize.minimize
result = minimize(nll, np.ones(2))
result

      fun: 1402.9894195313827
 hess_inv: array([[0.00094934, 0.00044141],
       [0.00044141, 0.00048183]])
      jac: array([0.00000000e+00, 1.52587891e-05])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 45
      nit: 12
     njev: 15
   status: 2
  success: False
        x: array([-0.07829136,  0.9841774 ])

In [3]:
# initialize Minuit with the fit result from scipy.optimize.minimize
m = Minuit(nll, result.x)
m.errordef = Minuit.LEAST_SQUARES
m.hesse()  # this also works without calling MIGRAD before

,Name,Value,Hesse Error,Minos Error-,Minos Error+,Limit-,Limit+,Fixed
0,x[0],-0.08,0.04,,,,,
1,x[1],0.984,0.031,,,,,


If the confidence intervals produced by HESSE are sufficient, iminuit can be used effectively together with other minimizers.

We can also compute the "Hesse errors" at any other point than the minimum. These cannot be interpreted as confidence intervals, they are just some numbers related to the second derivative of the cost function at that point.


In [4]:
m.values = (1.0, 0.5)
m.hesse()

,Name,Value,Hesse Error,Minos Error-,Minos Error+,Limit-,Limit+,Fixed
0,x[0],1.00,0.05,,,,,
1,x[1],0.500,0.009,,,,,


Using MINOS does not work (yet...), however. MINOS requires an existing function minimum produced by MIGRAD, which contains more information than just the parameter values.

In [5]:
try:
    m.minos()
except:
    import traceback

    traceback.print_exc()

Traceback (most recent call last):
  File "<ipython-input-5-8ebd7c3f1a25>", line 2, in <module>
    m.minos()
  File "/Users/hdembinski/Extern/iminuit/src/iminuit/_minuit.py", line 644, in minos
    raise RuntimeError(
RuntimeError: MINOS require function to be at the minimum. Run MIGRAD first.
